# UAS Data Science – Campus Recruitment\nStudi kasus: **Faktor Akademik yang Mempengaruhi Penempatan Kerja**\n\nNotebook ini berisi EDA, preprocessing, training model (LogReg & Random Forest), evaluasi, dan feature importance.

In [ ]:
import pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\n\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.preprocessing import OneHotEncoder, StandardScaler\nfrom sklearn.compose import ColumnTransformer\nfrom sklearn.pipeline import Pipeline\nfrom sklearn.impute import SimpleImputer\nfrom sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, roc_curve, auc\nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn.ensemble import RandomForestClassifier\n\nDATA_PATH = r'/mnt/data/Campus Recruitment.csv'\nOUT_DIR = r'/mnt/data/UAS_CampusRecruitment_GitHub'\nFIG_DIR = OUT_DIR + '/figures'\nimport os; os.makedirs(FIG_DIR, exist_ok=True)\n

In [ ]:
df = pd.read_csv(DATA_PATH)\ndf.head()\n

In [ ]:
TARGET = 'status kelulusan (Bekerja/Belum)'\nSALARY = 'Gaji'\ndf[TARGET].value_counts()\n

In [ ]:
# Persiapan data\nX = df.drop(columns=[TARGET, SALARY], errors='ignore').copy()\ny = df[TARGET].astype(str)\nif 'ID' in X.columns: X = X.drop(columns=['ID'])\ncat_cols = X.select_dtypes(include=['object']).columns.tolist()\nnum_cols = [c for c in X.columns if c not in cat_cols]\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)\n\nnumeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])\ncategorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore'))])\npreprocess = ColumnTransformer(transformers=[('num', numeric_transformer, num_cols), ('cat', categorical_transformer, cat_cols)])\n

In [ ]:
# Model\nlog_reg = Pipeline(steps=[('preprocess', preprocess), ('model', LogisticRegression(max_iter=2000, class_weight='balanced', random_state=42))])\nrf = Pipeline(steps=[('preprocess', preprocess), ('model', RandomForestClassifier(n_estimators=600, random_state=42, class_weight='balanced', n_jobs=-1))])\nlog_reg.fit(X_train, y_train)\nrf.fit(X_train, y_train)\n

In [ ]:
# Evaluasi cepat\ny_pred_lr = log_reg.predict(X_test)\nprint(classification_report(y_test, y_pred_lr, zero_division=0))\ncm_lr = confusion_matrix(y_test, y_pred_lr, labels=['Not Placed','Placed'])\nConfusionMatrixDisplay(cm_lr, display_labels=['Not Placed','Placed']).plot(values_format='d')\nplt.show()\n